Para esse caso de estudo, discutirei uma possível solução para o problema apresentado.

Alguns pontos observados inicialmente, ao olhar para o formato de dado usado como exemplo:

- O relatório/fonte de dados está armazenado no que assumo ser uma planilha de Excel/google sheets. Isso é um ponto positivo, pois a maior parte das ferramentas de importação de dados tem uma integração razoável com esse tipo de arquivo.
- Considero a fonte de dados como tendo uma certa estrutura, mesmo que o formato não seja o mais ideal para análises de dados.
- Temos alguns dados "soltos", que são relevantes àquela coleta de dados, mas não se encaixam em nenhuma tabela. Isso indica que precisaremos de uma classe de dados mais complexa, algo que armazene todos os detalhes presentes na metade superior do relatório, além das tabelas presentes na metade inferior.


Dadas essas observações, já é possível traçejar e implementar uma possível solução para o manuseio dos dados.

Antes disso, porém, gostaria de destacar alguns pontos que irei "assumir" para esse estudo de caso, em nome da objetividade e clareza da solução.

- Suposição 1: Todos as células presentes no relatório são fixas, isso é, entre um relatório e outro, não ocorrerá de uma descrição de dado e seu valor atrelado mudarem de lugar numa frequência que necessite de retrabalho constante.
- Suposição 2: Todos os relatórios são iguais. Além das células não mudarem de lugar, todos os relatórios possuirão as mesmas células e os mesmos tipos de dados contidos nelas. Dessa forma, é possível implementar uma solução genérica que funcione em qualquer relatório enviado.
- Suposição 3: Não é necessário separar as tabelas de movimentos/contagem de veículos. Pude notar por meio da análise prévia do relatório que existe uma separação entre os períodos de coleta (manhã, tarde e noite) e até entre os intervalos do mesmo período. O dado do intervalo em si é, claro, muito importante, mas é possível armazenar todos os períodos e intervalos em apenas uma estrutura de dados, sem impactar a capacidade de fazermos consultas a períodos e intervalos específicos. Para ser franco, é possível que tenha um impacto negativo na performance, pois teríamos tabelas maiores, mas é um custo ínfimo se comparado a praticidade e facilidade de implementação ao trabalharmos com apenas uma tabela.
- Suposição 4: Cada conjunto de colunas [Horário] + [Auto, Bus, Cam., Moto, Bici. UVP] corresponde a apenas 1 movimento. Para estruturação no formato de tabela, faremos com que o movimento seja mais uma coluna.

Após todas as observações e suposições detalhadas, podemos seguir adiante com os detalhes da solução:

1. O coração da solução serão os objetos do tipo dataframe, muito utilizados para representar dados colunares.
2. Comumente, a biblioteca de python "Pandas" é utilizada para manuseio de dataframes, mas para esse exercício, utilizarei a biblioteca "Polars" que é mais nova (e por isso menos "estável" e atestada, porém tem performance muito superior se comparada à "Pandas". Fonte: https://h2oai.github.io/db-benchmark/). Além disso, para importação direta de datas, será necessário plugar a lib "pyarrow" ao "Polars"
3. Como citado na etapa de observação, temos alguns dados que não cabem na visão de dataframe construída até aqui, como "Ponto de Pesquisa" ou "Endereço". É possível incorporar "data da pesquisa" na tabela, se for necessário, mas nesse ponto, não está clara a necessidade disso. Sendo assim, faz-se necessário uma classe que armazene os dados gerais e dataframe. Essa abordagem talvez não faça sentido num Python Notebook como esse, mas tem suas vantagens quando aplicada à produção de um projeto.
4. Será necessário um script que extraia todos os dados relevantes da planilha e os use para compor os objetos estruturados criados em Python. Para esse fim, a biblioteca "openpyxl" será usada. Ela permite que cada célula individual de um arquivo .xlsx seja acessada.

Para começar, instalamos e importamos as dependências necessárias.

In [1]:
# These extra bits are fail-safes to ensure the packages are being installed in
# the jupyter kernel that is actually, currently, running

import sys
!{sys.executable} -m pip install polars
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install pyarrow

import polars as pl
import openpyxl
from datetime import datetime

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Implementação do extrator de dados.

In [2]:
def xlsx_extractor(file_path: str):
    """
    Receives a .xlsx file path.
    return the data extracted from the sheet.    
    """
    def extract_hour_from_interval(date_interval: str):
        """
        Receives a string in the format:
        "hh:mm às hh:mm"
        Returns the 2 hours in datetime.time format.
        """
        start_time = date_interval[:5]
        end_time = date_interval[9:]

        start_time = datetime.strptime(start_time, '%H:%M').time()
        end_time = datetime.strptime(end_time, '%H:%M').time()

        return start_time, end_time

    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    # Getting general data

    research_point = sheet['B2'].value
    address = sheet['B3'].value
    research_date = sheet['B6'].value
    researh_date_weekday = sheet['C6'].value
    period_rows = [5, 8, 13]

    intervals_fhp = {}
    for period_row in period_rows:
        period = sheet.cell(row=period_row, column=1).value
        period = period[9:]

        row_range = 1
        if period == "Tarde":
            row_range = 3

        for i in range(row_range):
            index = i + 1
            interval = sheet.cell(row=period_row+index, column=5)
            interval_start, interval_end = extract_hour_from_interval(interval.value)
            fhp = sheet.cell(row=period_row+index, column=7).value
            key = (period, interval_start, interval_end)
            intervals_fhp[key] = fhp

    # Getting table data

    table_data = {
        "interval_start": [],
        "interval_end": [],
        "movimento": [],
        "Auto": [],
        "Bus": [],
        "Cam.": [],
        "Moto": [],
        "Bici.": [],
        "UVP": [],
    }

    # Iterate through rows and movimentos to get count data.

    movimentos_columns = [2, 8]

    for movimento_column in movimentos_columns:
        movimento = sheet.cell(row=16, column=movimento_column).value

        # getting interval separately
        for row in sheet.iter_rows(min_row=19, min_col=1, max_row=22, max_col=1, values_only=True):
            interval = row[0]
            interval_start, interval_end = extract_hour_from_interval(interval)
            table_data["interval_start"].append(interval_start)
            table_data["interval_end"].append(interval_end)

        min_col = movimento_column
        max_col = min_col+5

        for row in sheet.iter_rows(min_row=19, min_col=min_col, max_row=22, max_col=max_col, values_only=True):
            table_data["movimento"].append(int(movimento))
            table_data["Auto"].append(int(row[0]))
            table_data["Bus"].append(int(row[1]))
            table_data["Cam."].append(int(row[2]))
            table_data["Moto"].append(int(row[3]))
            table_data["Bici."].append(int(row[4]))
            table_data["UVP"].append(int(row[5]))

    columns = [
        ("interval_start", pl.Time),
        ("interval_end", pl.Time),
        ("movimento", pl.Int64),
        ("Auto", pl.Int64),
        ("Bus", pl.Int64),
        ("Cam.", pl.Int64),
        ("Moto", pl.Int64),
        ("Bici.", pl.Int64),
        ("UVP", pl.Float64),
    ]
    dataframe = pl.DataFrame(table_data, columns=columns)

    return {
        "research_point": research_point,
        "address": address,
        "research_date": research_date,
        "researh_date_weekday": researh_date_weekday,
        "intervals_fhp": intervals_fhp,
        "dataframe": dataframe
    }

testando extrator e visualizando o objeto dataframe criado.

In [3]:
file_path = "./case_study_assets/TPF-test.xlsx"
extracted_data = xlsx_extractor(file_path)
for key in extracted_data:
    if key != "dataframe":
        print(key, ":", extracted_data[key])

df = extracted_data['dataframe']
print(df.head())

research_point : 4.0
address : Rua tal
research_date : 2022-08-01 00:00:00
researh_date_weekday : Segunda
intervals_fhp : {('Manhã', datetime.time(7, 30), datetime.time(8, 30)): 0.87, ('Tarde', datetime.time(12, 0), datetime.time(13, 0)): 0.6, ('Tarde', datetime.time(14, 30), datetime.time(15, 30)): 0.67, ('Tarde', datetime.time(17, 0), datetime.time(18, 0)): 0.62, ('Noite', datetime.time(20, 0), datetime.time(21, 0)): 0.71}
shape: (5, 9)
┌────────────────┬──────────────┬───────────┬──────┬─────┬──────┬──────┬───────┬───────┐
│ interval_start ┆ interval_end ┆ movimento ┆ Auto ┆ ... ┆ Cam. ┆ Moto ┆ Bici. ┆ UVP   │
│ ---            ┆ ---          ┆ ---       ┆ ---  ┆     ┆ ---  ┆ ---  ┆ ---   ┆ ---   │
│ time           ┆ time         ┆ i64       ┆ i64  ┆     ┆ i64  ┆ i64  ┆ i64   ┆ f64   │
╞════════════════╪══════════════╪═══════════╪══════╪═════╪══════╪══════╪═══════╪═══════╡
│ 07:30:00       ┆ 07:45:00     ┆ 1         ┆ 131  ┆ ... ┆ 1    ┆ 16   ┆ 1     ┆ 154.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌